In [ ]:
import pandas as pd
import os

In [ ]:
def read_bank_csv(file_dir, dat_lst, *, usecols=None, skiprows=None, invertamount=False):
    """Load all csv file from a given folder, and return a list of dataframe. 
    Parameters:
        usecols : list of integer specifying which columns to read.
        skiprows: list of integer specifying rows to skip when loading csv.
        invertamount : boolean. If True, invert the "Amount" column. """
    flist = os.listdir(file_dir)
    fnames = [os.path.join(file_dir, f) for f in flist if f.endswith('.csv')]
    for fname in fnames:
        df = pd.read_csv(fname, names=['Date', 'Payee', 'Amount'], usecols=usecols, skiprows=skiprows)
        if invertamount:
            df['Amount'] = -1*df['Amount']
        dat_lst.append(df)
    return dat_lst


After downloading all bank statement, put all the ones you want to analyze into separate folders by financial institude. 

Note: some institude label spending negative, some don't. They also have different number of columns in the statement. Therefore statement from different institution can be put separately so they can be read separately by the read_bank_csv()

In [ ]:
bank1_dir = "C:\\path\\to\\your\\bank\\statement\\folder"
bank2_dir = ""
bank3_dir = ""


In [ ]:
dat_lst = []
dat_lst = read_bank_csv(bank1_dir, dat_lst, usecols=[0,1,3], skiprows=[0], invertamount=True)
dat_lst = read_bank_csv(bank2_dir, dat_lst, usecols=[0,1,2], skiprows=None, invertamount=False)
dat_lst = read_bank_csv(bank3_dir, dat_lst, usecols=[0,1,2], skiprows=[0], invertamount=False)
dat = pd.concat(dat_lst, ignore_index=True)
dat['Type'] = 'n/a'
dat.head()

In [ ]:
dat.tail()

Adjust payee categories. The shop name needs to match part of the full text in the Payee. 

In [ ]:
payee_types = {'Restaurant':['McDonald', 'Wendy', 'Happy Lamb', 'Shanghai', 
                             'KFC', 'Tim Hortons', 'Subway', 'coco', 'RAMEN',
                             'SHAWARMA', 'Tea', 'Starbucks', 'Wok hey', 
                             'INDIAN DESI','pizza', 'sushi','favoris', 'beavertails',
                             'mandarin','BOOSTER JUICE','NAYAX'],
               'Grocery': ['Costco', 'Farm Boy','Wal-Mart','Food Basic', 
                           'T&T','FRESHCO','green fresh', 'YIG', 'KOWLOON'],
               'Electronics': ['Best Buy', 'BBYMarket', 'OPEN BOX'],
               'Home': ['Ikea', 'CDN TIRE', 'HP', 'dollarama', 'SHoPPER +', 
                        'Temu.com', 'value village', 'staples'],
               'Service': ['Apple.com', 'domain.com', 'google', 'ufile', 'Consumerlab.com'],
               'Medical':['GORDON','Jennifer tan', 'GREAT MOUNTAIN GINSENG'],
               'Entertainment':['ticketmaster','funhaven','CINEPLEX',
                                'Canadian Museum', 'toy', 'zoo', '4029357733',
                                'bowling', 'aquarium', 'musee', 'JEEBA', 'mid earth',
                                ],
               'Travel':['VIA RAIL', 'MTO','BUDGET'],
               'Education':['Virtual ventures', 'lingoace', 'Music Fest', 'Tennis School', 
                            'Catholic school','Carleton','RCM', 'Kiwanis', 'bccma', 'zi chan',
                            'zichan', 'Fan Liu', 'mico basketball','RuiXue'],
               'Personal': ['Nu skin', 'thebay.com', 'hudson', 'EYEBUYDIRECT',
                            'Deciem','MARSHALLS', 'shoppers',"THE CHILDREN'S PLACE",
                            'PARTY CITY','linni Zhao'],
               'Donation': ['United way'],
               'Transportation': ['ESSO', 'Chrysler','petro', 'NAC', 'ALGONQUIN COLLEGE PARK', 'costco gas'],
               'Interest and Fee': ['Fee',],
               'Pet':['GREENBANK ANIMAL'],
               'Utilities': ['hydro', 'water', 'enbridge', 'bell', 'ITALK'],
               'Govn doc': ['NOTARY', ],
               'Business trip': ['AIR-SERV', 'PENTICTON', 'LA PREP','YOW' ,'TAXI', 'LAKESIDE VILLA']}


Go through the dataframe and assign type to all the transaction

In [ ]:
for i, row in dat.iterrows():
    for type, shops in payee_types.items():
        for s in shops:
            if s.upper() in row['Payee'].upper():
                dat.loc[i,'Type'] = type

Take a look what other payees that are not assigned. 
If you loaded debit card statement, you could have lots of credit card payment, and pay check deposit without type. This is intended, as I don't include those in my analysis. 

In [ ]:
dat[dat['Type']=='n/a']['Payee'].unique()

Summarize expenses, excluding types that are 'n/a'

In [ ]:
dat_typed = dat.loc[dat['Type']!='n/a', :]
(dat_typed.groupby('Type').sum()).sort_values('Amount', ascending=False)

If you want to see data from specific retailer, use this cell: 

In [ ]:
substring='Costco'
filter = dat['Payee'].str.contains(substring, case=False)
dat_filt = dat[filter]
dat_filt